In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sts

Vamos a crear un dataset con puntuaciones de usuarios:

In [2]:
df = pd.DataFrame({'item': ['manzana', 'sandia', 'pera', 'pizza', 'perrito', 'hamburguesa',
                            'ensalada', 'hummus', 'chuleton', 'longaniza', 'lubina'],
                   'a': [5, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0],
                   'b': [4, 3, 0, 5, 4, 4, 0, 0, 4, 4, 0],
                   'c': [4, 3, 3, 0, 0, 0, 3, 4, 2, 0, 2],
                   'd': [1, 2, 1, 5, 5, 5, 0, 2, 4, 3, 0],
                   'e': [5, 5, 5, 0, 0, 0, 4, 5, 0, 0, 0],
                   'f': [5, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0]})
df.set_index('item', inplace=True)
df

,a,b,c,d,e,f
item,,,,,,
manzana,5,4,4,1,5,5
sandia,5,3,3,2,5,5
pera,0,0,3,1,5,0
pizza,0,5,0,5,0,0
perrito,0,4,0,5,0,0
hamburguesa,0,4,0,5,0,0
ensalada,4,0,3,0,4,4
hummus,0,0,4,2,5,5
chuleton,0,4,2,4,0,0


Y ahora otro que indica si el usuario compró o no el producto:

In [3]:
df_bin = (df >0).astype(int)
df_bin

,a,b,c,d,e,f
item,,,,,,
manzana,1,1,1,1,1,1
sandia,1,1,1,1,1,1
pera,0,0,1,1,1,0
pizza,0,1,0,1,0,0
perrito,0,1,0,1,0,0
hamburguesa,0,1,0,1,0,0
ensalada,1,0,1,0,1,1
hummus,0,0,1,1,1,1
chuleton,0,1,1,1,0,0


Para medir la afinidad entre usuarios y así poder recomendar, se utilizan los conjuntos matemáticos y sus INTERSECCIONES

In [4]:
#Conjuntos en Python:
A = set([1, 2, 4])
B = set([2, 4, 6])
A.intersection(B)

{2, 4}

In [5]:
A.union(B)

{1, 2, 4, 6}

In [6]:
#Cardinalidad: 
len(A.intersection(B))

2

¿Qué es lo que hace a dos usuarios iguales o similares y otros muy distintos? Similitud de Jaccard (ver notas cuaderno)

In [7]:
#Hacemos una función para la similitud de Jaccard:

jaccard = lambda a, b: len(a.intersection(b)) / len(a.union(b))

In [8]:
jaccard(set([1,2,3]), set([1,3,4]))

0.5

In [9]:
jaccard(set([1,2,3]), set([1,3,2]))

1.0

In [11]:
#Creamos conjuntos con los alimentos que ha comprado el usuario a:
set(df_bin[df_bin['a'] >0].index)

{'ensalada', 'manzana', 'sandia'}

In [14]:
#Hallamos las distancias entre un usuario y todos los demás (distancia de Jaccard):
for u0 in df_bin.columns:
    for u1 in df_bin.columns:
        if u0 != u1:
            items0 = set(df_bin[df_bin[u0] > 0].index)
            items1 = set(df_bin[df_bin[u1] > 0].index)
            print('sim({}, {}) = {}' .format(u0,u1,jaccard(items0,items1)))

sim(a, b) = 0.25
sim(a, c) = 0.42857142857142855
sim(a, d) = 0.2
sim(a, e) = 0.6
sim(a, f) = 0.75
sim(b, a) = 0.25
sim(b, c) = 0.2727272727272727
sim(b, d) = 0.7777777777777778
sim(b, e) = 0.2
sim(b, f) = 0.2222222222222222
sim(c, a) = 0.42857142857142855
sim(c, b) = 0.2727272727272727
sim(c, d) = 0.45454545454545453
sim(c, e) = 0.7142857142857143
sim(c, f) = 0.5714285714285714
sim(d, a) = 0.2
sim(d, b) = 0.7777777777777778
sim(d, c) = 0.45454545454545453
sim(d, e) = 0.4
sim(d, f) = 0.3
sim(e, a) = 0.6
sim(e, b) = 0.2
sim(e, c) = 0.7142857142857143
sim(e, d) = 0.4
sim(e, f) = 0.8
sim(f, a) = 0.75
sim(f, b) = 0.2222222222222222
sim(f, c) = 0.5714285714285714
sim(f, d) = 0.3
sim(f, e) = 0.8


Vamos a escoger a los K-vecinos más parecidos a mi, y en base a lo que hayan comprado ellos, me recomendaré lo que yo todavía no haya consumido.

In [15]:
u0 = 'a'
neighbors = []

for u1 in df_bin.columns:
    if u0 != u1:
        items0 = set(df_bin[df_bin[u0] > 0].index)
        items1 = set(df_bin[df_bin[u1] > 0].index)
        print('sim({}, {}) = {}' .format(u0,u1,jaccard(items0,items1)))
        neighbors.append((u1, jaccard(items0, items1)))

sim(a, b) = 0.25
sim(a, c) = 0.42857142857142855
sim(a, d) = 0.2
sim(a, e) = 0.6
sim(a, f) = 0.75


In [16]:
neighbors

[('b', 0.25), ('c', 0.42857142857142855), ('d', 0.2), ('e', 0.6), ('f', 0.75)]

In [20]:
top = sorted(neighbors, key=lambda x: -x[1])[:2]

In [21]:
top

[('f', 0.75), ('e', 0.6)]

In [23]:
#item -> score
recommendations = {}

for neighbor, similarity in top:
    for item in df_bin[df_bin[neighbor] > 0].index:
        if item not in set(df_bin[df_bin['a'] > 0].index):
            if item in recommendations:
                recommendations[item] +=similarity
            else:
                recommendations[item] = similarity

In [24]:
recommendations

{'hummus': 1.35, 'pera': 0.6}

Para el caso de las valoraciones (ratings), utilizaremos mejor las correlaciones de Pearson:

In [26]:
u0 = 'a'
for u1 in df.columns:
    if u0 != u1:
        print(u0, u1, sts.pearsonr(df[u0],df[u1]))

a b (-0.014025268244765073, 0.9673537305875948)
a c (0.5630164352612619, 0.07133112524015639)
a d (-0.47773066916197693, 0.13724307333312027)
a e (0.6402913757205406, 0.033822514649123954)
a f (0.789823242704962, 0.0038279681529224444)


El coeficiente de correlación es el que aparece primero, y el p-valor es el segundo (éste hay que minimizarlo).
Los más similares son, por tanto, a y f.

In [27]:
%pwd

'/home/ainhoa/Master/Jupyter notebooks Python'